# **Desenvolvimento de Data-Driven Apps com Python**
# **Teste de Performance 3 (TP3)**

## Instituto Infnet - Rafael Dottori de Oliveira

### 05/12/2024

---

*Neste TP3, você deve demonstrar sua capacidade de integrar ferramentas, agentes e raciocínio interativo para resolver problemas complexos utilizando Large Language Models (LLMs).*

*O desafio envolve criar uma aplicação funcional que combine os conceitos de ReAct Agents, memória conversacional e ferramentas externas (tools), com criatividade e originalidade.*

*• Entrega*

*Submeta o repositório do projeto no GitHub até a data estipulada.*

*Garanta que o repositório seja público ou acessível.*

*• Dica:*

*Use sua criatividade para propor um problema inovador e explore ao máximo as funcionalidades dos agentes com LLMs!*

---

## **Exercício 1: Defina o Problema**

*Escolha um problema que possa ser resolvido com o uso de agentes inteligentes.*

*Descreva o objetivo da aplicação, o público-alvo e as principais funcionalidades planejadas.*

*Detalhe os casos de uso que serão testados para validar a solução.*

Vamos criar um assistente sobre cinema.

O objetivo é criar um modelo que consiga pesquisar as informações de um filme, criar um resumo do enredo e gerar análise de sentimentos das opiniões da audiência.

Faremos testes com filmes existentes e verificando se a memória conversacional do modelo funciona.

---

## **Exercício 2: Configure o Framework LangChain**

*Configure o ambiente de desenvolvimento para utilizar LangChain, escolhendo um LLM apropriado (como Gemini ou GPT-4) para sua aplicação.*

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os


api_key = api_key=os.getenv('GEMINI_KEY')

modelo = ChatGoogleGenerativeAI(model='gemini-1.5-flash', api_key=api_key)

In [2]:
modelo.invoke('Olá, tudo bom?').content

'Olá! Tudo ótimo, e você?\n'

---

## **Exercício 3: Implemente Ferramentas**

*Crie e configure no mínimo duas ferramentas externas (tools). Cada ferramenta deve ter nome, descrição e função claramente definidos, e ser integrada ao LangChain.*

In [ ]:
from langchain.agents import Tool


def analise_sentimento(avaliacao):
    instrucoes = f"""
    Analise o seguinte texto e determine se o sentimento geral é positivo, neutro ou negativo.
              
    • Avaliação:
    {avaliacao}
    """
    
    return modelo.invoke(instrucoes).content


tool_analise_sentimento = Tool(
    name='Análise de Sentimento',
    func=analise_sentimento,
    description='Analisar se uma opinião sobre um filme é positiva, neutra ou negativa.'
)

In [ ]:
analise_sentimento('Esse filme é excelente. Apesar de que o final deixa a desejar.')

'O sentimento geral é **positivo**, apesar da ressalva sobre o final.  A palavra "excelente" domina a avaliação. A crítica ao final diminui o positivo, mas não o inverte para negativo.\n'

In [29]:
def resumir_enredo(filme):
    instrucoes = f"""
    Resuma o enredo do seguinte filme, sem entregar surpresas do final do filme:

    {filme}
    """

    return modelo.invoke(instrucoes).content

tool_resumir_enredo = Tool(
    name='Resumir Enredo',
    func=resumir_enredo,
    description='Resume o enredo de um filme.'
)

In [30]:
resumir_enredo('Titanic')

'Titanic conta a história de Jack Dawson, um artista pobre, e Rose DeWitt Bukater, uma jovem rica noiva de um homem rico e arrogante.  Eles se encontram a bordo do navio RMS Titanic durante sua viagem inaugural de Southampton a Nova York. Apesar de suas diferentes classes sociais, eles se apaixonam.  Seu romance floresce em meio ao luxo e à opulência do navio, mas é ameaçado pela rígida estrutura social da época e pelo inevitável desastre que se aproxima.\n'

---

## **Exercício 4: Utilize o Framework ReAct**

*Implemente o raciocínio interativo usando o framework ReAct, garantindo que o agente possa alternar entre raciocinar, tomar ações e observar os resultados para alcançar os objetivos.*

In [31]:
from langchain.agents import initialize_agent

agente = initialize_agent(
    tools=[tool_analise_sentimento, tool_resumir_enredo],
    llm=modelo,
    agent='zero-shot-react-description',
    verbose=True
)

In [32]:
agente.run('Me dê a sinopse do filme Baby Driver. Avalie as opiniões dos usuários.')



> Entering new AgentExecutor chain...
Thought: Para responder à pergunta, preciso primeiro resumir o enredo de Baby Driver e depois analisar o sentimento das opiniões dos usuários sobre o filme.  Vou usar as ferramentas em sequência.

Action: Resumir Enredo
Action Input: Baby Driver

Observation: Baby é um jovem motorista de fuga talentoso que usa a música para bloquear o ruído e o caos ao seu redor enquanto executa trabalhos perigosos para um chefe do crime. Ele anseia por uma vida melhor e tenta se libertar do seu mundo criminoso, mas é pressionado por dívidas e ameaças de violência, o que o força a fazer mais um trabalho que pode ser seu último.  Sua tentativa de escapar da vida que leva o envolve em uma teia de traições e perseguições de tirar o fôlego.

Thought:Thought: Agora que tenho o resumo do enredo, preciso analisar o sentimento das opiniões dos usuários sobre o filme.  Vou usar a ferramenta de análise de sentimento com algumas opiniões de exemplo, assumindo que tenho aces

'O filme Baby Driver conta a história de um jovem motorista de fuga que usa a música para lidar com o estresse de seus trabalhos perigosos para um chefe do crime. Ele tenta escapar desse mundo, mas é forçado a realizar mais um trabalho que pode ser seu último, envolvendo-se em traições e perseguições.  As opiniões dos usuários sobre o filme são mistas, porém, inclinam-se levemente para o lado positivo, com avaliações entusiásticas superando as negativas e neutras.'

---

## **Exercício 5: Adicione Memória Conversacional**

*Configure um sistema de memória conversacional, permitindo que o agente lembre o histórico de interações e utilize esse contexto para gerar respostas mais coerentes.*

In [34]:
from langchain.memory import ConversationBufferMemory

memoria = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

C:\Users\Dottori\AppData\Local\Temp\ipykernel_16408\2376323519.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memoria = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [37]:
from langchain.agents import initialize_agent

agente = initialize_agent(
    tools=[tool_analise_sentimento, tool_resumir_enredo],
    llm=modelo,
    agent='conversational-react-description',
    verbose=True,
    memory=memoria
)

In [38]:
resposta_1 = agente.run('Me diga qual a história do primeiro Star Wars')

resposta_2 = agente.run('Agora faça a análise de sentimentos do filme')

resposta_3 = agente.run('Qual o filme que pedi na 1ª pergunta?')



> Entering new AgentExecutor chain...
```tool_code
Thought: Do I need to use a tool? Yes
Action: Resumir Enredo
Action Input: Star Wars: Episódio IV - Uma Nova Esperança
Observation: Em uma galáxia distante, a tirânica e todo-poderosa força imperial governa a galáxia com punho de ferro.  Um grupo rebelde luta para se libertar do domínio imperial.  Princesa Leia, um membro chave da Rebelião, rouba planos secretos da Estrela da Morte, uma arma de destruição planetária, e os envia para fora.  Ela é capturada, mas os planos caem nas mãos de Luke Skywalker, um jovem fazendeiro, e Obi-Wan Kenobi, um misterioso e poderoso cavaleiro Jedi.  Juntos, eles embarcam em uma missão perigosa para resgatar a Princesa Leia e destruir a Estrela da Morte, com a ajuda de Han Solo, um contrabandista, e seu copiloto Chewbacca.  Sua jornada os leva a enfrentar os perigos da Galáxia e a descoberta de uma força maior que os conecta.

Thought:```tool_code
Thought: Do I need to use a tool? No
AI: A história de 

---

## **Exercício 6: Desenvolva a Lógica do Agente**

*Estruture a lógica de funcionamento do agente, integrando ferramentas, memória e raciocínio interativo para resolver o problema proposto.*

In [ ]:
from langchain.agents import Tool, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
import os


def analise_sentimento(avaliacao):
    instrucoes = f"""
    Analise o seguinte texto e determine se o sentimento geral é positivo, neutro ou negativo.
              
    • Avaliação:
    {avaliacao}
    """
    
    return modelo.invoke(instrucoes).content


def resumir_enredo(filme):
    instrucoes = f"""
    Resuma o enredo do seguinte filme, sem entregar surpresas do final do filme:

    {filme}
    """

    return modelo.invoke(instrucoes).content


api_key = api_key=os.getenv('GEMINI_KEY')
modelo = ChatGoogleGenerativeAI(model='gemini-1.5-flash', api_key=api_key)

tool_analise_sentimento = Tool(
    name='Análise de Sentimento',
    func=analise_sentimento,
    description='Analisar se uma opinião sobre um filme é positiva, neutra ou negativa.'
)

tool_resumir_enredo = Tool(
    name='Resumir Enredo',
    func=resumir_enredo,
    description='Resume o enredo de um filme.'
)

memoria = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

agente = initialize_agent(
    tools=[tool_analise_sentimento, tool_resumir_enredo],
    llm=modelo,
    agent='conversational-react-description',
    verbose=True,
    memory=memoria
)

---

## **Exercício 7: Projete a Interface do Usuário**

*Crie uma interface funcional (streamlit ou linha de comando) para que o agente possa ser utilizado. Não é necessário que seja sofisticada, mas deve ser clara e intuitiva.*

In [ ]:
%%writefile app_streamlit.py

import streamlit as st
from langchain.agents import Tool, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
import os


def analise_sentimento(avaliacao):
    instrucoes = f"""
    Analise o seguinte texto e determine se o sentimento geral é positivo, neutro ou negativo.
              
    • Avaliação:
    {avaliacao}
    """
    
    return modelo.invoke(instrucoes).content


def resumir_enredo(filme):
    instrucoes = f"""
    Resuma o enredo do seguinte filme, sem entregar surpresas do final do filme:

    {filme}
    """

    return modelo.invoke(instrucoes).content


api_key = api_key=os.getenv('GEMINI_KEY')
modelo = ChatGoogleGenerativeAI(model='gemini-1.5-flash', api_key=api_key)

tool_analise_sentimento = Tool(
    name='Análise de Sentimento',
    func=analise_sentimento,
    description='Analisar se uma opinião sobre um filme é positiva, neutra ou negativa.'
)

tool_resumir_enredo = Tool(
    name='Resumir Enredo',
    func=resumir_enredo,
    description='Resume o enredo de um filme.'
)

if 'memoria' not in st.session_state:
    st.session_state['memoria'] = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

agente = initialize_agent(
    tools=[tool_analise_sentimento, tool_resumir_enredo],
    llm=modelo,
    agent='conversational-react-description',
    verbose=True,
    memory=st.session_state['memoria']
)

st.set_page_config(page_title='TP3 - Filmes com LangChain', page_icon='🎬')

st.title('Filmes com LangChain')

avatares = {
'human': 'user',
'ai': 'assistant'
}

if st.sidebar.button('Limpar histórico de mensagens'):
    st.session_state['memoria'].chat_memory.clear()

for mensagem in st.session_state['memoria'].chat_memory.messages:
    st.chat_message(avatares[mensagem.type]).write(mensagem.content)

if prompt := st.chat_input('Digite sua mensagem'):
    st.chat_message('user').write(prompt)
    with st.spinner('Processando...'):
        try:
            resposta = agente.run(prompt)
            st.chat_message('assistant').write(resposta)
        except:
            st.error('Erro ao gerar a resposta.')

Overwriting app_streamlit.py


---

## **Exercício 8: Teste a Solução**

*Realize testes práticos utilizando os casos de uso definidos. Inclua uma explicação clara sobre como o agente facilita ou possibilita o trabalho e o que seria necessário fazer sem o uso desse recurso.*

Os agentes facilitam as buscas que seriam feitas sobre as informações do filmes e automatizam tanto na criação da sinopse quanto na avaliação das opiniões dos usuários.

Abaixo mostramos um teste da memória conversacional e da rapidez em que o modelo nos traz as informações solicitadas.

![teste](Testes.png)

---

## **Exercício 9: Documente o Projeto**

*Submeta o código no GitHub com um README bem elaborado. O README deve incluir:*

*Descrição do problema e da solução.*

*Casos de uso testados e resultados observados.*

*Instruções claras para execução do código.*

*Reflexão sobre como o agente facilitou ou possibilitou o trabalho em comparação a uma abordagem manual.*

## **Exercício 10: Boas Práticas de Codificação**

*Certifique-se de que o código segue boas práticas de desenvolvimento, incluindo:*

*Estrutura modular e organizada.*

*Comentários explicativos e concisos.*

*Nomes de variáveis e funções claros e descritivos.*

*Tratamento adequado de erros.*